### Bogdan Bintu
### Copyright Presidents and Fellows of Harvard College, 2018.

### Standard analysis of the STORM data

The subfolders H#R# indicate the hybrdization round and the index of the 30kb region imaged respectively.

The region chr21:28Mb-29.2Mb was imaged with 30Kb resolution using STORM across two independent datasets with 100-150 chromosomes each.

Each subfolder contains both diffraction-limited and STORM imaged fields of view, each corresponding to a '.dax' file with the fluorescent signal recorded by the camera. 
This file format can be opened with tools in ChromatinImaging\CommonTools\IOTools.py

#### Imaging experimental details:

For the first round of hybridization in a multiplexed STORM imaging experiment we mixed into the hybridization buffer the following oligonucleotides: 1) the first 30-nt Alexa647-labeled readout probe (at 100 nM) which has homology to the readout sequence on the primary probes targeting the first 30 kb of chromatin (chr21:28000001-28030000) and 2) an Alexa405-labeled activator probe (at 100 nM).

For the subsequent rounds we added: 1) an Alexa647-labeled readout probe (100 nM) targeting the readout sequence on the primary probes hybridized to the next 30-kb segment and 2) an unlabeled oligo sequence (1 μM) fully complementary to the previously flowed readout probe. The unlabeled oligo is used to remove the previous readout probe as detailed in the toehold displacement reaction.

We used the first 41 rounds of sequential hybridization and imaging of readout probes to target the 41 consecutive 30-kb chromatin segments in the genomic region of interest. After this, we used additional rounds to relabel some of the readout probes for assessing alignment accuracy and sample variation over time. We performed three-dimensional difraction-limited and then STORM imaging after each round of hybridization to each chromatin segment. 

Imaging of ~18 fields of view for the 41 segments took approximately 5 days in total. 

The 3-valve fluidics system that we constructed allowed for the loading of only 20 different hybridization solutions. Consequently, after every 20 rounds we short circuited the fluidics chamber, washed each of the 20 channels with 30% formamide in water and loaded new hybridization solutions. 




#### Script used for STORM analysis
#### Each notebook sections should be performed sequentially. Note: some parts require GUI input.

In [2]:
#External imports
import numpy as np
import os
#Basic functions are located here:
import GeneralToolsSTORMTracing as gtct

## Part 1
### Get a list of all .dax files corresponding to STORM movies residing in the first hybe folder.
### Start a series of GUIs which allow selecting the chromosomal positions based on the Cy3 signal in the first hybe of the STORM movies.

### Naming convention:
### A master folder contains subfolders labelled H#iR#j.  (hybe round #i, region #j)
### The subfolders contain matching STORM and difraction-limited imaging files labelled STORM_#k.dax and Conv_zscan_#k.dax where #k indicates the field of view number.

In [1]:
%matplotlib notebook

In [ ]:
#### Part 1
##Get a list of all .dax files corresponding to STORM movies residing in the first hybe folder.
##Start a series of GUIs which allow selecting the chromosomal positions based on the Cy3 signal in the first hybe of the STORM movies.

first_hybe_folder = r""
list_daxes = gtct.listDax(first_hybe_folder,name='STORM_')
dic_dax = [gtct.STORM_STD_pointSelect(dax_fl) for dax_fl in list_daxes[:]]

In [ ]:
## Decide on a master_save_folder and save the selected positions
master_save_folder = r""
save_folder = master_save_folder+os.sep+'selectedSpots'
if not os.path.exists(save_folder): os.makedirs(save_folder)
big_dic={}
for dic in dic_dax:
    big_dic.update(dic)
for dax_fl in big_dic.keys():
    imshow_obj = big_dic[dax_fl]
    #save coords
    dic_name = gtct.save_name_dic(dax_fl,save_folder=master_save_folder)
    pickle.dump(imshow_obj.coords,open(dic_name["coords_file"],'w'))
    #save figure
    imshow_obj.fig.set_size_inches(12, 12)
    imshow_obj.fig.savefig(dic_name["png_figure"], dpi=100)

## Part 2
#### Once a round of hybridization and imaging is complete the corresponding subfolder H#iR#j folder is copied automatically from the solid state drive to one of 3 8Tb drives available.
#### Get pointers to these files and start aligning them to pixel precision.

In [ ]:
drive_folders = [r'drive1_folder',
                 r'drive2_folder',
                 r'drive3_folder']
#List all the dax
dax_files=[]
for data_folder in drive_folders:
    dax_files += gtct.listDax(data_folder,name='STORM_')
#Next partition according to field of view and sort according to the order of hybridization
fovs = [gtct.extract_flag(dax_file,'STORM_','.') for dax_file in dax_files]
def sort_to_hybe(dax_fls):
    r'given a list of dax files containing the hybe folder info \H<index>R...\it returns an sorted by index list of dax files'
    map_=[int(dax_fl.split(os.sep+'H')[-1].split('R')[0]) for dax_fl in dax_fls]
    return list(np.array(dax_fls)[np.argsort(map_)])
dax_files_fovs = map(sort_to_hybe,gtct.partition_map(dax_files,fovs))

#save file names for convenience in a subfolder called roughAlingment
save_folder = master_save_folder+os.sep+'roughAlingment'
if not os.path.exists(save_folder): os.makedirs(save_folder)
pickle.dump(dax_files_fovs,open(save_folder+os.sep+'dax_files_fovs.pkl','w'))

## Perform rough registration and save results.
ind_ref = 0 #The hybe index to align to. By default everything is aligned to the first hybe.
dic_registration={} 
coords_dic={}
refs_dic={}

# If failed or decide to restart load previously saved results.
dic_registration_fl = save_folder+os.sep+'dic_registration.pkl'
if os.path.exists(dic_registration_fl):
    dic_registration=pickle.load(open(dic_registration_fl,'r'))
    coords_dic=pickle.load(open(save_folder+os.sep+'coords_dic.pkl','r'))
    refs_dic=pickle.load(open(save_folder+os.sep+'refs_dic.pkl','r'))

for cy3_dax_files_fov in dax_files_fovs:
    nm0 = cy3_dax_files_fov[ind_ref]
    im0 = gtct.load_cy3(nm0,func=np.max)
    for nm1 in cy3_dax_files_fov[:]:
        basenm0 = "--".join(nm0.split(os.sep)[-2:])
        basenm1 = "--".join(nm1.split(os.sep)[-2:])
        key_nm = basenm0+'<-'+basenm1
        print key_nm
        #if not dic_registration.has_key(key_nm):
        if not coords_dic.has_key(nm1):
            im1 = gtct.load_cy3(nm1,func=np.max)
            txy = gtct.fftalign_guess(im0,im1,center=[0,0],max_disp=50) # this is the main function.  Register everyone to the ref field of view using fft correlation.
            dic_registration[key_nm] = txy # collect to dictionary

            coord_file = gtct.save_name_dic(nm0,master_save_folder)["coords_file"]
            if os.path.exists(coord_file):
                coords = pickle.load(open(coord_file,'r'))
                coords_dic[nm1]=np.array(coords)-np.array([txy[::-1]])
                refs_dic[nm1] = np.array(coords)
            else:
                coords_dic[nm1] = coords_dic[nm0]-np.array([txy[::-1]])
                refs_dic[nm1] = refs_dic[nm0]
            #save figure
            fig = plt.figure(figsize=(12,12))
            ax = fig.add_subplot(111)
            xy = zip(*coords_dic[nm1])
            if len(xy)>0:
                ax.plot(np.array(xy[0])-1,np.array(xy[1])-1, 'o',markersize=12,markeredgewidth=1,markeredgecolor='g',markerfacecolor='None')
                for i_txt,(x_,y_) in enumerate(coords_dic[nm1]):
                    ax.text(x_-1,y_-1,str(i_txt),color='g')
            ax.imshow(im1,cmap=cm.hot,interpolation='nearest')

            file_ = "--".join(nm1.split(os.sep)[-2:][::-1]).replace('.dax','')+'_cy3-roughalignment.png'
            fig.savefig(save_folder+os.sep+file_)
            plt.close(fig)
            #save dics
            pickle.dump(coords_dic,open(save_folder+os.sep+'coords_dic.pkl','w'))
            pickle.dump(refs_dic,open(save_folder+os.sep+'refs_dic.pkl','w'))
            pickle.dump(dic_registration,open(dic_registration_fl,'w'))

## Part 3
### Crop data to selected chromosomes
### Note: The cropping is very slow even with memorymapping. Hard limitation due to the nature of HDD. Croping from the SSD is faster but SSD drives of large size are expensive. Automatic croping during aquisition would be more elegant. However it is more risky due to possible registration errors.

In [ ]:
#load relevant data
master_save_folder = r''
save_folder = master_save_folder+os.sep+'roughAlingment'
# load list of all storm dax files grouped according to field of view
dax_files_fovs = pickle.load(open(save_folder+os.sep+'dax_files_fovs.pkl','r'))
# load the registration dictionaries
dic_registration = pickle.load(open(save_folder+os.sep+'dic_registration.pkl','r'))
coords_dic = pickle.load(open(save_folder+os.sep+'coords_dic.pkl','r'))
refs_dic = pickle.load(open(save_folder+os.sep+'refs_dic.pkl','r'))

save_folder = master_save_folder+os.sep+'crop'

#Partition according to drive and start cropping.
dax_files_fovs_drive = gtct.partition_map(gtct.flatten(dax_files_fovs),[fl[0] for fl in gtct.flatten(dax_files_fovs)])

## Multiple hard drives can be cropped at the same time. 
## Durring aquisition folders are copied distributively across separete hard drives. Thus the cropping from multiple hard drives could be started in parallel.
index_drive = 0
dax_files = dax_files_fovs_drive[index_drive]
def sort_val(fl):
    hindex_2digits = '{:0>2}'.format(fl.split(os.sep+'H')[-1].split('R')[0])
    fov_index = fl.split(os.sep+'STORM_')[-1].split('.dax')[0]
    return hindex_2digits+'_'+fov_index
    
ind_sort = np.argsort(map(sort_val,dax_files))
dax_files = list(np.array(dax_files)[ind_sort])

failed=[]

for dax_fl in dax_files:
    print "Dealing with: "+str(dax_fl)
    coords_ = np.array(coords_dic[dax_fl],dtype=int)
    refs_ = np.array(refs_dic[dax_fl],dtype=int)
    start = time.time()
    try:
        gtct.STORM_STD_crop(dax_fl,coords_,save_folder=save_folder,s_good=50,rep_fr = 50,memmap=False,tag=map(str,map(list,refs_)),overwrite=False)
    except:
        failed.append(dax_fl)
    print "Elapsed time: "+str(time.time()-start)
print "Failed cropping files:"
print failed

## Part 4
#### Copy conventional imaging files to local save folder. These are small compared to the STORM.

In [ ]:
master_save_folder = r""
save_folder = master_save_folder+os.sep+'roughAlingment'
dax_files_fovs = pickle.load(open(save_folder+os.sep+'dax_files_fovs.pkl','r'))

folders = np.unique([os.path.dirname(fl) for fl in gtct.flatten(dax_files_fovs)])
list_all_files=[]
for folder in folders:
    for root, dirs, files in os.walk(folder, topdown=False):
        for nm in files:
            list_all_files.append(os.path.join(root, nm))
list_files = np.setdiff1d(list_all_files,gtct.flatten(dax_files_fovs)) #exclude the STORM files
sizes = [os.path.getsize(fl) for fl in list_files]
print "Size of conventional files: " + str(np.sum(sizes)/10.**9) + " GB"

#prepare folders for copying
master_conv_folder = master_save_folder+os.sep+'ConvEmptyStorm'
files_i = list_files[:]
files_f = [master_conv_folder+os.sep+os.sep.join(fl.split(os.sep)[-2:]) for fl in files_i]
for fl in files_f:
    folder = os.path.dirname(fl)
    if not os.path.exists(folder):
        os.makedirs(folder)
files_i_cp,files_f_cp=[],[]
for fl1,fl2 in zip(files_i,files_f):
    if not os.path.exists(fl2):
        files_i_cp.append(fl1)
        files_f_cp.append(fl2)

#copy!
import copyFilesWindows as cfw      
cfw.copy(files_i_cp,files_f_cp)#This could be replaced with shutil copying.

## Part 5
#### Start batch fitting the cropped STORM files on multiple processors.

In [ ]:
import StormAnalysisAdditions as saa

master_save_folder = r""
save_folder = master_save_folder+os.sep+'roughAlingment'
dax_files = glob.glob(r'F:\Bogdan_Analysis\10_25_2017_CDSH30k-rep5\crop'+os.sep+'*_storm.dax')
def sort_by_fov_and_hybe(fl):
    hindex_2digits = '{:0>2}'.format(fl.split('--H')[-1].split('R')[0].split('A')[0])
    fov_index = fl.split('--STORM_')[-1].split('_')[0]
    return fov_index+'_'+hindex_2digits
ind_sort = np.argsort(map(sort_by_fov_and_hybe,dax_files))
dax_files = list(np.array(dax_files)[ind_sort])
dax_files = [fl for fl in dax_files if not os.path.exists(fl.replace('.dax','_mlist.bin'))] #do not overwrite
bin_files = [fl.replace('.dax','_mlist.bin') for fl in dax_files]
parm_file = r'fit3DStorm.xml'#this is the paramater file used for fitting. 
#More details on this in the ZhuangLab/storm_analysis git repository
#The z paramaters match the PSF of the STORM setup
parms_files = [parm_file for fl in dax_files]
saa.fitFrameBatch(dax_files, bin_files, parms_files, over_write=1, mock=False, batch_size_=50)
#Note: This batch fitting is performed on a 20-core CPU machine with 256Gb of RAM. Adjust the batch_size acording to your system.